In [1]:
import os
import random
import time
import numpy as np
import numpy as _np
if not hasattr(_np, 'complex'):
    _np.complex = complex

import librosa
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv1D, ZeroPadding1D,
    Activation, Multiply, Add,
    Softmax, Layer
)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout

2025-04-27 13:31:17.497883: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745760679.897041     934 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745760680.537385     934 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-27 13:31:26.830165: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

SAMPLE_RATE  = 16000 
MAX_DURATION = 3.0                      
MAX_SAMPLES  = int(SAMPLE_RATE * MAX_DURATION) 
BATCH_SIZE   = 8                     
EPOCHS       = 50                      


# load and assign labels

In [3]:
TARGET_EMOTIONS = ['neutral', 'sad', 'happy', 'angry']

In [4]:
EMO_MAP = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

def load_ravdess(path):
    X, y = [], []
    for fp in glob(os.path.join(path, "Actor_*", "*.wav")):
        code = os.path.basename(fp).split('-')[2]
        label = EMO_MAP.get(code)
        if label not in TARGET_EMOTIONS:
            continue
        sig, _ = librosa.load(fp, sr=SAMPLE_RATE)
        sig, _ = librosa.effects.trim(sig, top_db=20)
        if len(sig) < MAX_SAMPLES:
            sig = np.pad(sig, (0, MAX_SAMPLES - len(sig)), 'constant')
        else:
            sig = sig[:MAX_SAMPLES]
        X.append(sig)
        y.append(label)
    return np.array(X), np.array(y)


In [5]:
CREMA_EMO_MAP = {
    "ANG": "angry",
    "DIS": "disgust",
    "FEA": "fearful",
    "HAP": "happy",
    "NEU": "neutral",
    "SAD": "sad"
}


def load_crema(path, limit=3500):
    X, y = [], []
    files = glob(os.path.join(path, "*.wav"))
    count = 0
    for fp in files:
        if count >= limit:
            break
        filename = os.path.basename(fp)
        parts = filename.split('_')
        emo_code = parts[2]
        label = CREMA_EMO_MAP.get(emo_code)
        if label not in TARGET_EMOTIONS:
            continue
        sig, _ = librosa.load(fp, sr=SAMPLE_RATE)
        sig, _ = librosa.effects.trim(sig, top_db=20)
        if len(sig) < MAX_SAMPLES:
            sig = np.pad(sig, (0, MAX_SAMPLES - len(sig)), 'constant')
        else:
            sig = sig[:MAX_SAMPLES]
        X.append(sig)
        y.append(label)
        count += 1
    return np.array(X), np.array(y)


In [6]:
RAVDESS_PATH = '../data/RAVDESS_Data'
CREMA_PATH   = '../data/CREMA_Data'

print("Loading & trimming RAVDESS...")
X_ravdess, y_ravdess = load_ravdess(RAVDESS_PATH)
print(f"Loaded {X_ravdess.shape[0]} samples from RAVDESS")

print("Loading & trimming CREMA-D...")
X_crema, y_crema = load_crema(CREMA_PATH)
print(f"Loaded {X_crema.shape[0]} samples from CREMA-D")

Loading & trimming RAVDESS...
Loaded 672 samples from RAVDESS
Loading & trimming CREMA-D...
Loaded 3500 samples from CREMA-D


In [7]:

X_total = np.concatenate([X_ravdess, X_crema])
y_total = np.concatenate([y_ravdess, y_crema])

print(f"Tổng số samples sau khi gộp: {X_total.shape[0]}")

X_total, y_total = shuffle(X_total, y_total, random_state=42)

lb = LabelBinarizer()
y_onehot = lb.fit_transform(y_total)


X_tr, X_va, y_tr, y_va = train_test_split(
    X_total, y_onehot, test_size=0.2, stratify=y_total, random_state=42
)

X_tr = X_tr[..., np.newaxis]
X_va = X_va[..., np.newaxis]

print(f"Train set: {X_tr.shape}, Validation set: {X_va.shape}")
print(f"Số lượng class: {len(lb.classes_)} - {lb.classes_}")

Tổng số samples sau khi gộp: 4172
Train set: (3337, 48000, 1), Validation set: (835, 48000, 1)
Số lượng class: 4 - ['angry' 'happy' 'neutral' 'sad']


# Model architecture

In [8]:
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)


class AttentionPooling(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.score_conv = Conv1D(1, 1, padding='same', name="attn_score_conv")
        self.softmax    = Softmax(axis=1, name="attn_weights")

    def call(self, inputs):
        score   = self.score_conv(inputs)
        weights = self.softmax(score)
        return tf.reduce_sum(weights * inputs, axis=1)


def conv1d_causal(x, filters, kernel_size, dilation_rate, name):
    pad = (kernel_size - 1) * dilation_rate
    x   = ZeroPadding1D((pad, 0), name=f"{name}_pad")(x)
    return Conv1D(filters, kernel_size,
                  dilation_rate=dilation_rate,
                  padding='valid',
                  name=name)(x)


def dilation_block(x, res_ch, dil_ch, skip_ch, k_size, rate, name):
    
    f = conv1d_causal(x, dil_ch, k_size, rate, name=f"{name}_f")
    f = Activation('tanh', name=f"{name}_tanh")(f)
    g = conv1d_causal(x, dil_ch, k_size, rate, name=f"{name}_g")
    g = Activation('sigmoid', name=f"{name}_sigmoid")(g)
    z = Multiply(name=f"{name}_gate")([f, g])
    skip = Conv1D(skip_ch, 1, padding='same', name=f"{name}_skip")(z)
    res  = Conv1D(res_ch,   1, padding='same', name=f"{name}_res")(z)
    x    = Add(name=f"{name}_out")([res, x])
    return x, skip


def build_wavenet_ser(time_steps, n_classes,
                                res_ch=32, dil_ch=32, skip_ch=64,
                                k_size=2, dilation_rates=None):
    if dilation_rates is None:
        dilation_rates = [2**i for i in range(8)]

    inp = Input(shape=(time_steps,1), name="input_wave")
    x   = Conv1D(res_ch, 1, padding='same', name="pre_conv")(inp)

    skips = []
    for i, rate in enumerate(dilation_rates, 1):
        x, s = dilation_block(
            x, res_ch, dil_ch, skip_ch,
            k_size, rate,
            name=f"dil{i}_r{rate}"
        )
        skips.append(s)

    x = Add(name="skip_sum")(skips)
    x = Activation('relu', name="post_relu")(x)
    x = Conv1D(skip_ch, 1, activation='relu', name="post_conv1")(x)
    x = Dropout(0.3, name="post_dropout")(x)  
    x = Conv1D(n_classes, 1, name="post_conv2")(x)

    context = AttentionPooling(name="attn_pool")(x)
    out     = Activation('softmax', name="softmax")(context)

    return Model(inp, out, name="WaveNet_SER")


model = build_wavenet_ser(
    MAX_SAMPLES, len(lb.classes_)
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


model.summary()

I0000 00:00:1745762272.553054     934 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "WaveNet_SER"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_wave          │ (None, 48000, 1)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pre_conv (Conv1D)   │ (None, 48000, 32) │         64 │ input_wave[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dil1_r1_f_pad       │ (None, 48001, 32) │          0 │ pre_conv[0][0]    │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dil1_r1_g_pad       │ (None, 48001, 32) │          0 │ pre_conv[0][0]    │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dil1_r1_f (Conv1D)  │ (None, 48000, 32) │      2,080 │ dil1_r1_f_pad[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dil1_r1_g (Conv1D)  │ (None, 48000, 32) │      2,080 │ dil1_r1_g_pad[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dil1_r1_tanh        │ (None, 48000, 32) │          0 │ dil1_r1_f[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dil1_r1_sigmoid     │ (None, 48000, 32) │          0 │ dil1_r1_g[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dil1_r1_gate        │ (None, 48000, 32) │          0 │ dil1_r1_tanh[0][… │
│ (Multiply)          │                   │            │ dil1_r1_sigmoid[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dil1_r1_res         │ (None, 48000, 32) │      1,056 │ dil1_r1_gate[0][… │
│ (Conv1D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dil1_r1_out (Add)   │ (None, 48000, 32) │          0 │ dil1_r1_res[0][0… │
│                     │                   │            │ pre_conv[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dil2_r2_f_pad       │ (None, 48002, 32) │          0 │ dil1_r1_out[0][0] │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dil2_r2_g_pad       │ (None, 48002, 32) │          0 │ dil1_r1_out[0][0] │
│ (ZeroPadding1D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dil2_r2_f (Conv1D)  │ (None, 48000, 32) │      2,080 │ dil2_r2_f_pad[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dil2_r2_g (Conv1D)  │ (None, 48000, 32) │      2,080 │ dil2_r2_g_pad[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dil2_r2_tanh        │ (None, 48000, 32) │          0 │ dil2_r2_f[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dil2_r2_sigmoid     │ (None, 48000, 32) │          0 │ dil2_r2_g[0][0]   │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dil2_r2_gate        │ (None, 48000, 32) │          0 │ dil2_r2_tanh[0][… │
│ (Multiply)          │                   │            │ dil2_r2_sigmoid[… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 62,057 (242.41 KB)

 Trainable params: 62,057 (242.41 KB)

 Non-trainable params: 0 (0.00 B)

# Training


In [9]:

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=1e-6),
    tf.keras.callbacks.ModelCheckpoint('best_wavenet_ser.h5', monitor='val_loss', save_best_only=True, verbose=1),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')   
]


start_time = time.time()

history = model.fit(
    X_tr, y_tr,
    validation_data=(X_va, y_va),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=2
)

end_time = time.time()
print(f"\nTraining completed in {(end_time - start_time)/60:.2f} minutes.")


loss, acc = model.evaluate(X_va, y_va, batch_size=BATCH_SIZE)
print(f"\nFinal validation accuracy: {acc*100:.2f}%")

2025-04-27 06:38:47.053959: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 640704000 exceeds 10% of free system memory.
2025-04-27 06:38:58.294234: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 640704000 exceeds 10% of free system memory.


Epoch 1/50


I0000 00:00:1745735956.380080   37436 service.cc:148] XLA service 0x7f435c002260 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745735956.384122   37436 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-04-27 06:39:17.232042: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1745735958.871981   37436 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-27 06:39:44.547789: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'loop_transpose_fusion_30', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1745735984.610362   37436 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-04-27 06:41:51.794169: I


Epoch 1: val_loss improved from inf to 0.98615, saving model to best_wavenet_ser.h5


418/418 - 184s - 441ms/step - accuracy: 0.4924 - loss: 1.1302 - val_accuracy: 0.5665 - val_loss: 0.9862 - learning_rate: 1.0000e-03
Epoch 2/50

Epoch 2: val_loss improved from 0.98615 to 0.97153, saving model to best_wavenet_ser.h5


...

418/418 - 120s - 287ms/step - accuracy: 0.6083 - loss: 0.9009 - val_accuracy: 0.5940 - val_loss: 0.9182 - learning_rate: 3.0000e-04
Epoch 16/50

Epoch 16: val_loss improved from 0.91822 to 0.91418, saving model to best_wavenet_ser.h5


418/418 - 120s - 288ms/step - accuracy: 0.6125 - loss: 0.8953 - val_accuracy: 0.5964 - val_loss: 0.9142 - learning_rate: 3.0000e-04
Epoch 17/50

Epoch 17: val_loss did not improve from 0.91418
418/418 - 120s - 287ms/step - accuracy: 0.6173 - loss: 0.8896 - val_accuracy: 0.5940 - val_loss: 0.9172 - learning_rate: 3.0000e-04
Epoch 18/50

Epoch 18: val_loss did not improve from 0.91418
418/418 - 120s - 287ms/step - accuracy: 0.6185 - loss: 0.8840 - val_accuracy: 0.5904 - val_loss: 0.9174 - learning_rate: 3.0000e-04
Epoch 19/50

Epoch 19: val_loss did not improve from 0.91418
418/418 - 120s - 286ms/step - accuracy: 0.6227 - loss: 0.8791 - val_accuracy: 0.5916 - val_loss: 0.9248 - learning_rate: 3.0000e-04
Epoch 20/50

Epoch 20: val_loss improved from 0.91418 to 0.90727, saving model to best_wavenet_ser.h5


418/418 - 120s - 288ms/step - accuracy: 0.6290 - loss: 0.8657 - val_accuracy: 0.5904 - val_loss: 0.9073 - learning_rate: 9.0000e-05
Epoch 21/50

Epoch 21: val_loss did not improve from 0.90727
418/418 - 121s - 290ms/step - accuracy: 0.6353 - loss: 0.8612 - val_accuracy: 0.5916 - val_loss: 0.9085 - learning_rate: 9.0000e-05
Epoch 22/50

Epoch 22: val_loss did not improve from 0.90727
418/418 - 120s - 286ms/step - accuracy: 0.6368 - loss: 0.8572 - val_accuracy: 0.5892 - val_loss: 0.9094 - learning_rate: 9.0000e-05
Epoch 23/50

Epoch 23: val_loss did not improve from 0.90727
418/418 - 120s - 287ms/step - accuracy: 0.6410 - loss: 0.8546 - val_accuracy: 0.5892 - val_loss: 0.9110 - learning_rate: 9.0000e-05
Epoch 24/50

Epoch 24: val_loss did not improve from 0.90727
418/418 - 120s - 287ms/step - accuracy: 0.6500 - loss: 0.8461 - val_accuracy: 0.5820 - val_loss: 0.9358 - learning_rate: 2.7000e-05
Epoch 25/50

Epoch 25: val_loss did not improve from 0.90727
418/418 - 120s - 287ms/step - accur

2025-04-27 07:32:05.618893: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 160320000 exceeds 10% of free system memory.


105/105 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 0.5757 - loss: 0.8982

Final validation accuracy: 59.04%


In [12]:
model.save('wavenet_ser_model.keras')

# Đánh Giá Mô Hình Chạy Đơn (Single Run Evaluation)

In [2]:
import time
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Đo thời gian bắt đầu đánh giá
start_eval = time.time()

# Dự đoán trên tập Validation
y_pred = model.predict(X_va, batch_size=BATCH_SIZE)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_va, axis=1)

# Tính toán các chỉ số
accuracy  = accuracy_score(y_true_classes, y_pred_classes) * 100
precision = precision_score(y_true_classes, y_pred_classes, average='macro') * 100
recall    = recall_score(y_true_classes, y_pred_classes, average='macro') * 100
f1        = f1_score(y_true_classes, y_pred_classes, average='macro') * 100

# Classification Report & Confusion Matrix
report = classification_report(y_true_classes, y_pred_classes, target_names=lb.classes_)
cm     = confusion_matrix(y_true_classes, y_pred_classes)

end_eval = time.time()
eval_time = end_eval - start_eval

# In kết quả
print(f"Accuracy      : {accuracy:.2f}%")
print(f"Precision     : {precision:.2f}%")
print(f"Recall        : {recall:.2f}%")
print(f"F1-score      : {f1:.2f}%")
print(f"Evaluation Time: {eval_time:.2f} seconds\n")

print("=== Classification Report ===")
print(report)

print("=== Confusion Matrix ===")
print(cm)


NameError: name 'model' is not defined

In [1]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mse = mean_squared_error(y_true_classes, y_pred_classes)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true_classes, y_pred_classes)
r2 = r2_score(y_true_classes, y_pred_classes)

NameError: name 'y_true_classes' is not defined

# Đánh Giá Bằng K-Fold Cross Validatio


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import time


n_splits = 5
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)

# Lưu kết quả từng fold
acc_list, pre_list, rec_list, f1_list, time_list = [], [], [], [], []

# Bắt đầu vòng lặp K-Fold
for fold, (train_idx, val_idx) in enumerate(kf.split(X_total, y_total), 1):
    print(f"\n===== Fold {fold} / {n_splits} =====")
    
    # Lấy dữ liệu cho fold hiện tại
    X_tr, X_va = X_total[train_idx][..., None], X_total[val_idx][..., None]
    y_tr, y_va = y_onehot[train_idx], y_onehot[val_idx]

    # Khởi tạo lại mô hình cho mỗi fold
    model = build_wavenet_ser(
        time_steps=MAX_SAMPLES,
        n_classes=len(lb.classes_)
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    start_time = time.time()

    model.fit(
        X_tr, y_tr,
        batch_size=BATCH_SIZE,
        epochs=20,            
        verbose=0
    )
    train_time = time.time() - start_time


    y_pred = model.predict(X_va, batch_size=BATCH_SIZE)
    y_pred_cls = np.argmax(y_pred, axis=1)
    y_true_cls = np.argmax(y_va, axis=1)

    acc = accuracy_score(y_true_cls, y_pred_cls) * 100
    pre = precision_score(y_true_cls, y_pred_cls, average='macro') * 100
    rec = recall_score(y_true_cls, y_pred_cls, average='macro') * 100
    f1s = f1_score(y_true_cls, y_pred_cls, average='macro') * 100

    # Lưu kết quả
    acc_list.append(acc)
    pre_list.append(pre)
    rec_list.append(rec)
    f1_list.append(f1s)
    time_list.append(train_time)

    print(f"Fold {fold}: Acc={acc:.2f}%, Precision={pre:.2f}%, Recall={rec:.2f}%, F1={f1s:.2f}%, Time={train_time:.2f}s")


print("\n === K-Fold Summary ===")
print(f"Accuracy : {np.mean(acc_list):.2f}% ± {np.std(acc_list):.2f}")
print(f"Precision: {np.mean(pre_list):.2f}% ± {np.std(pre_list):.2f}")
print(f"Recall   : {np.mean(rec_list):.2f}% ± {np.std(rec_list):.2f}")
print(f"F1-score : {np.mean(f1_list):.2f}% ± {np.std(f1_list):.2f}")
print(f"Avg Time : {np.mean(time_list):.2f} seconds")



===== Fold 1 / 5 =====


2025-04-27 13:58:09.972445: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 640704000 exceeds 10% of free system memory.
2025-04-27 13:58:11.257243: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 640704000 exceeds 10% of free system memory.
I0000 00:00:1745762303.835170    6455 service.cc:148] XLA service 0x7f38d8003ec0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745762303.841642    6455 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-04-27 13:58:24.573786: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1745762306.008945    6455 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-27 13:58:46.906678: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Re

105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 61ms/step
Fold 1: Acc=55.81%, Precision=59.08%, Recall=55.47%, F1=55.51%, Time=2354.59s

===== Fold 2 / 5 =====


2025-04-27 14:37:43.313948: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 640704000 exceeds 10% of free system memory.
2025-04-27 14:37:48.284149: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 640704000 exceeds 10% of free system memory.
2025-04-27 14:38:04.346661: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'loop_transpose_fusion_30', 4 bytes spill stores, 4 bytes spill loads

2025-04-27 14:40:06.031128: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'loop_transpose_fusion_9', 4 bytes spill stores, 4 bytes spill loads



105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step
Fold 2: Acc=58.32%, Precision=60.90%, Recall=59.02%, F1=57.72%, Time=2347.75s

===== Fold 3 / 5 =====


...

105/105 ━━━━━━━━━━━━━━━━━━━━ 14s 67ms/step
Fold 3: Acc=58.39%, Precision=60.45%, Recall=57.57%, F1=57.41%, Time=2345.26s

===== Fold 4 / 5 =====


2025-04-27 15:58:55.090231: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'loop_transpose_fusion_30', 4 bytes spill stores, 4 bytes spill loads



105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 54ms/step
Fold 4: Acc=63.43%, Precision=64.48%, Recall=62.88%, F1=63.30%, Time=2347.30s

===== Fold 5 / 5 =====


2025-04-27 16:38:26.616432: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'loop_transpose_fusion_30', 4 bytes spill stores, 4 bytes spill loads



105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 52ms/step
Fold 5: Acc=59.23%, Precision=59.68%, Recall=59.32%, F1=59.14%, Time=2352.87s

 === K-Fold Summary ===
Accuracy : 59.04% ± 2.48
Precision: 60.92% ± 1.89
Recall   : 58.85% ± 2.43
F1-score : 58.62% ± 2.61
Avg Time : 2349.55 seconds
